In [1]:
import tensorflow as tf
import numpy as np
import tensorflow.keras as k
import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import Adadelta, RMSprop,SGD,Adam
from tensorflow.python.client import device_lib

config = tf.compat.v1.ConfigProto(log_device_placement=True,
      gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.9))
      
sess = tf.compat.v1.Session(config=config)
physical_devices = tf.config.list_physical_devices('GPU') 
try:
  tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
  # Invalid device or cannot modify virtual devices once initialized.
  print('failed to set growth')

K.set_image_data_format('channels_first');

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 2070 SUPER, pci bus id: 0000:41:00.0, compute capability: 7.5



In [2]:
data = np.load(r"../TrainingData/data.npz")
x_train = data['n_frames']
print(x_train[0].shape)

(6, 8, 128, 128)


In [6]:
from GAN import GAN
try: del gan
except: pass
gan = GAN(channels=5, do_batch_norm=False,use_noise=True)
gan.compile(loss='binary_crossentropy', optimizer = Adam(learning_rate=.01),run_eagerly=True);
gan.build([50,5,8,128,128]);
gan.summary()

Model: "gan_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Generator (SimpleUGen)       multiple                  1458491   
_________________________________________________________________
Discriminator (discriminator multiple                  36073     
_________________________________________________________________
concatenate_1 (Concatenate)  multiple                  0         
Total params: 1,494,568
Trainable params: 1,494,564
Non-trainable params: 4
_________________________________________________________________


In [4]:
#from Discriminator import discriminator
#disc = discriminator(channels=5)
#disc.build([50,5,8,128,128]);
#disc.summary()

In [7]:
gan.fit(x_train,x_train,epochs=5,batch_size=5,shuffle=True)

Epoch 1/5
191/191 [==============================] - 28s 146ms/step - loss: 58.7192 - generator_tricking_metric: 189.0000 - discriminator_failing_metric: 341.0000 - bce_metric: 1.9948
Epoch 2/5
191/191 [==============================] - 28s 145ms/step - loss: 76532.5845 - generator_tricking_metric: 24.0000 - discriminator_failing_metric: 28.0000 - bce_metric: 0.4719
Epoch 3/5
191/191 [==============================] - 28s 144ms/step - loss: 127609.1837 - generator_tricking_metric: 2.0000 - discriminator_failing_metric: 14.0000 - bce_metric: 0.1182
Epoch 4/5
191/191 [==============================] - 28s 145ms/step - loss: 271444.8759 - generator_tricking_metric: 0.0000e+00 - discriminator_failing_metric: 1.0000 - bce_metric: 0.0054
Epoch 5/5
191/191 [==============================] - 28s 144ms/step - loss: 279252.0957 - generator_tricking_metric: 0.0000e+00 - discriminator_failing_metric: 0.0000e+00 - bce_metric: 0.0000e+00
